In [1]:
from faker import Faker
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_synthetic_rcm_data(n_visits=1000000, max_procs_per_visit=4):
    fake = Faker()
    carriers = [f"Carrier{i}" for i in range(1, 20)]
    procedure_params = {
        '99213': {'prob': 0.5, 'charge_amt': 120, 'allowed_range': (80, 100)},
        '99214': {'prob': 0.3, 'charge_amt': 150, 'allowed_range': (100, 120)},
        '93000': {'prob': 0.2, 'charge_amt': 200, 'allowed_range': (150, 170)}
    }
    codes = list(procedure_params.keys())
    weights = [procedure_params[c]['prob'] for c in codes]

    # Build per-carrier allowed ranges
    allowed_ranges = {
        carrier: {
            code: tuple(sorted([
                random.uniform(*procedure_params[code]['allowed_range']),
                random.uniform(*procedure_params[code]['allowed_range'])
            ]))
            for code in codes
        }
        for carrier in carriers
    }

    # Denial parameters
    denial_rate = 0.20
    denial_group_weights = {
        'eligibility': 0.25,
        'coding': 0.25,
        'benefits': 0.20,
        'timely_filing': 0.15,
        'other': 0.15
    }
    denial_codes_by_group = {
        'eligibility':   {'CARC1': 0.6, 'CARC2': 0.4},
        'coding':        {'CARC3': 0.7, 'CARC4': 0.3},
        'benefits':      {'CARC5': 1.0},
        'timely_filing': {'CARC6': 0.5, 'CARC7': 0.5},
        'other':         {'CARC8': 0.5, 'CARC9': 0.5}
    }

    start_date = datetime(2024, 1, 1)
    records = []

    for visit_idx in range(n_visits):
        visit_id = f"{100000 + visit_idx}"
        svc_date = (start_date + timedelta(days=random.randint(0, 545))).date()
        payer = random.choice(carriers)
        physician = fake.name()
        clinic = random.choice(['Northside Clinic', 'Downtown Health', 'Lakeside Medical'])

        # Determine number of procedures for this visit (distinct codes)
        n_procs = random.randint(1, min(max_procs_per_visit, len(codes)))
        procs = random.sample(codes, k=n_procs)

        for code in procs:
            charge_amt = procedure_params[code]['charge_amt']
            lo, hi = allowed_ranges[payer][code]
            allowed_amt = random.uniform(lo, hi)

            # Denial logic
            is_denied = random.random() < denial_rate
            if is_denied:
                group = random.choices(
                    list(denial_group_weights.keys()),
                    weights=list(denial_group_weights.values()), k=1
                )[0]
                dc_w = denial_codes_by_group[group]
                denial_code = random.choices(
                    list(dc_w.keys()), weights=list(dc_w.values()), k=1
                )[0]
                paid_amt = 0.0
                appeal_success = random.random() < 0.3
            else:
                denial_code = None
                paid_amt = random.uniform(allowed_amt * 0.8, allowed_amt)
                appeal_success = None

            adjust_amt = charge_amt - paid_amt

            records.append({
                'visit_id':       visit_id,
                'procedure_code': code,
                'svc_date':       svc_date,
                'payer_company':  payer,
                'payer_type':     'Commercial',
                'clinic':         clinic,
                'physician':      physician,
                'charge_amt':     charge_amt,
                'allowed_amt':    round(allowed_amt, 2),
                'adjust_amt':     round(adjust_amt, 2),
                'paid_amt':       round(paid_amt, 2),
                'denial_code':    denial_code,
                'appeal_success': appeal_success
            })

    return pd.DataFrame(records)

if __name__ == "__main__":
    df = generate_synthetic_rcm_data(n_visits=1000000, max_procs_per_visit=4)
    print(df.head(15))


   visit_id procedure_code    svc_date payer_company  payer_type  \
0    100000          93000  2025-04-21     Carrier17  Commercial   
1    100000          99213  2025-04-21     Carrier17  Commercial   
2    100001          99214  2024-03-19     Carrier12  Commercial   
3    100002          99214  2024-06-11     Carrier12  Commercial   
4    100002          99213  2024-06-11     Carrier12  Commercial   
5    100003          99214  2024-01-24     Carrier15  Commercial   
6    100003          99213  2024-01-24     Carrier15  Commercial   
7    100004          93000  2024-07-03     Carrier15  Commercial   
8    100005          93000  2025-06-06     Carrier17  Commercial   
9    100005          99213  2025-06-06     Carrier17  Commercial   
10   100006          99214  2024-12-07     Carrier15  Commercial   
11   100006          93000  2024-12-07     Carrier15  Commercial   
12   100007          99213  2024-02-22      Carrier9  Commercial   
13   100008          99214  2025-03-08     Carri

In [2]:
df

,visit_id,procedure_code,svc_date,payer_company,payer_type,clinic,physician,charge_amt,allowed_amt,adjust_amt,paid_amt,denial_code,appeal_success
0,100000,93000,2025-04-21,Carrier17,Commercial,Northside Clinic,Mr. Jeffrey Shepherd,200,158.39,200.00,0.00,CARC9,False
1,100000,99213,2025-04-21,Carrier17,Commercial,Northside Clinic,Mr. Jeffrey Shepherd,120,89.78,120.00,0.00,CARC5,False
2,100001,99214,2024-03-19,Carrier12,Commercial,Lakeside Medical,Anna Carter MD,150,106.39,51.96,98.04,None,None
3,100002,99214,2024-06-11,Carrier12,Commercial,Northside Clinic,Mark Wallace,150,105.81,65.09,84.91,None,None
4,100002,99213,2024-06-11,Carrier12,Commercial,Northside Clinic,Mark Wallace,120,88.00,120.00,0.00,CARC7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000382,1099996,99214,2024-08-28,Carrier14,Commercial,Downtown Health,Tracy Flynn,150,111.13,47.87,102.13,None,None
2000383,1099997,99214,2025-02-25,Carrier15,Commercial,Downtown Health,James Turner,150,116.45,150.00,0.00,CARC8,True
2000384,1099997,99213,2025-02-25,Carrier15,Commercial,Downtown Health,James Turner,120,92.81,37.05,82.95,None,None
2000385,1099998,99214,2024-05-20,Carrier1,Commercial,Northside Clinic,Denise Jordan,150,102.65,57.06,92.94,None,None


### Testing

### Test 2

Test3
